# 单类型 lennard_jones MD

## Import part

In [ ]:
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from PIL import Image
import glob
import moviepy.editor as mp
from datetime import datetime
import time
import os.path
from os import path
import shutil
from IPython.display import display
import random

In [ ]:
#输入文件(可自定义):
#dt = 0.005      #时间步长
#density =0.8       #流体密度
#initUcell_x = 20   #方格的x大小
#initUcell_y = 20   #y大小
#initUcell_z = 20
#stepAvg = 100       #用来抽样统计步数
#stepEquil = 0     #平衡步数
#num_steps = 500   #模拟循环数
#T = 1.   #温度设置

## Definition of mol and prop

In [2]:
class Mol():
    def __init__(self,pos,vel,a):
        """
        每一分子对象（mol）所具有的属性如下：
        pos:原子坐标
        vel:原子速度
        a:原子加速度向量
        """
        #初始化以上属性的值
        self.pos=np.asarray([0.0,0.0,0.0])
        self.vel=np.asarray([0.0,0.0,0.0])
        self.a=np.asarray([0.0,0.0,0.0])
        
# 体系属性类，主要用来求体系属性（测定量）的平均值和标准差        
class Prop():
   def __init__(self, val, sum, sum_squ ):
        self.val=val                       #体系测定量的值
        self.sum=sum                       #体系测定量的值的和
        self.sum_squ=sum_squ               #体系测定量的值的平方和

## Toolfuntions

In [5]:
import numpy as np
import random

random.uniform(0,1)

0.5102077194496979

In [ ]:
# 返回平方和立方值的函数：
def Sqr(x):
    return (x * x)


def Cube(x):
    return ((x) * (x) * (x))



# 用于生成 (0,1) 范围内的均匀分布的随机数
##def RandR():
    global randSeedP
    randSeedP=(randSeedP * IMUL + IADD) & MASK
    return (randSeedP*SCALE)## ?

# 用于在三维空间产生具有均匀分布的随机方向的单位向量
def VRand(p):
    s: float
    phi: float
    s = 2. * math.pi * random.uniform(0, 1)
    phi = 2. * math.pi * random.uniform(0, 1)
    p[0] = math.sin(s)*math.cos(phi)      # x-component of vector p
    p[1] = math.sin(s)*math.sin(phi)      # y-component of vector p
    p[2] = math.cos(s)                    # z-component of vector p
    return p



# 周期性边界（PBC）算法：
"""
第一部分如下：
体系内粒子的坐标的约束：
  三维体系中，一个方体元胞四周有相同的元胞(称为"镜像")，
  若一个粒子从右边边界穿出，操作上左边边界对应位置有相同粒子进入，
  由此保证了元胞中粒子数目即物理量如动量与能量的守恒。
  因此有以下结论：
  region[0]为x方向的元胞的边长，则粒子在x方向的坐标区间为：[-0.5*region[0],0.5*region[0]]
  regino[1]为y方向的元胞的边长, 则粒子在x方向的坐标区间为：[-0.5*region[1],0.5*region[1]]
  regino[2]为z方向的元胞的边长, 则粒子在x方向的坐标区间为：[-0.5*region[2],0.5*region[2]]

  以x方向元胞为例：假设边长为1x1x1的正方体元胞 
  当粒子坐标为x>=0.5时，为了确保当前粒子在一个1x1元胞中，有关系：x-新位置=1，
  故有：新位置=x-1
  同理得： x<-0.5时，有关系：新位置-（x）=1  故有:新位置=1+x，
  
第二部分：对粒子间距离计算使用最小镜像法则，见：ComputerForces()函数
"""
#PBC算法第一部分：对体系内粒子坐标的约束
def VWrapAll(v):
    #对元胞x方向的坐标约束
    if v[0]>=0.5*region[0]:    
        v[0]-=region[0]
    elif v[0]<-0.5*region[0]:
         v[0] +=region[0]
    
    #对元胞y方向的坐标约束
    if v[1] >= 0.5 * region[1]:
        v[1] -= region[1]
    elif v[1] < -0.5 * region[1]:
        v[1] += region[1]

    #对元胞z方向的坐标约束
    if v[0]>=0.5*region[2]:    
        v[0]-=region[2]
    elif v[0]<-0.5*region[2]:
         v[0] +=region[2]

## Main program

In [ ]:
mov = 1                                      # 如果你想做一个视频的话，设置 mov=1
workdir = str(os.getcwd()+'/')              # 为所有png和视频设置一个工作目录

# 如果/LJP目录不存在，就建立它，否则就删除/LJP（和它的内容）
# 创建一个新的/LJP目录。
if path.exists(str(workdir+'LJP'))==False:
    os.makedirs(str(workdir+'LJP'))
else:
    shutil.rmtree(str(workdir+'LJP'))
    os.makedirs(str(workdir+'LJP'))

# 加载输入参数文件
df_params = pd.read_csv('LJP.csv', sep=',', header=None, names=['parameter', 'value'])

#初始该模拟系统热力学属性值
NDIM = 3                        #维度设置
vSum = np.asarray([0.0, 0.0, 0.0])   #速度之和
kinEnergy =Prop(0.0, 0.0, 0.0)  #动能
totEnergy =Prop(0.0, 0.0, 0.0)  #势能
pressure  =Prop(0.0, 0.0, 0.0)  #压强

systemParams = []              #初始化系统参数列表

#用于产生随机数算法函数的变量
IADD = 453806245
IMUL = 314159269
MASK = 2147483647
SCALE = 0.4656612873e-9
randSeedP = 17

#初始化参数：把LJP.csv文件中的参数值传递给MD模拟所需要的相关变量
dt = float(df_params.values[0][1])              #模拟步长
density = float(df_params.values[1][1])             #体系密度


initUcell = np.asarray([0.0, 0.0, 0.0])            #初始化元胞，具体大小由输入文件密度决定
initUcell[0] = int(df_params.values[2][1])         #元胞X方向长度=20
initUcell[1] = int(df_params.values[3][1])         #元胞y方向长度=20
initUcell[2] = int(df_params.values[4][1])         #元胞z方向长度=20
 
stepAvg = int(df_params.values[5][1])              #抽样平均数
stepEquil = float(df_params.values[6][1])          #平衡步长
stepLimit = float(df_params.values[7][1])          #限制的步数
T = float(df_params.values[8][1])        #体系温度

#定义了一个Mol类对象的数组，大小为20*20*20=8000. mol[0]~mol[7999]，共7999个氩原子分子
mol = [Mol(np.asarray([0.0, 0.0, 0.0]), \
           np.asarray([0.0, 0.0, 0.0]), \
           np.asarray([0.0, 0.0, 0.0])) for i in range(int(initUcell[0]*initUcell[1]*initUcell[2]))]

global nMol        #定义分子的数量变量
nMol = len(mol)    #为mol数组的长度为8000

# 氩-氩相互作用的LJ势参数:
epsilon =  1
sigma = 1

# 执行体系初始化相关功能函数
SetParams()                                   #设置主程序内部模拟所需参数
SetupJob()                                    #初始化粒子的坐标，速度，加速
moreCycles = 1                                #MD循环控制开关变量1：run; 0:stop

n = 0                                        #记录步数，用来给每一步输出的坐标图命名                
while moreCycles:                            #MD循环
    SingleStep()                             #求解每一步的运动方程
    if mov==1:
        plotMolCoo(mol, workdir, n)          #制作单个步长的粒子坐标图
    n += 1
    if stepCount >= stepLimit:
        moreCycles = 0
        
#输出模拟过程中生成的参数
columns = ['timestep','timeNow', '$\Sigma v$', 'E', '$\sigma E$', 'Ek', '$\sigma Ek$', 'P_1', 'P_2']
df_systemParams = pd.DataFrame(systemParams, columns=columns)   

#绘制表格，仅在jupyter notebook使用，打印出数据列表
display(df_systemParams) 

if mov==1:                     
    makeMov()                 # 制作视频
GraphOutput()                 #输出体系测定量随模拟时间变化的图像

## Functions related to initilation

In [ ]:
"""
模拟所需的其他变量，不包括那些构成输入数据的变量，
由函数SetParams设置。
"""
def SetParams():
    global rCut    #截断距离
    global region  #区域
    global velMag  #速度变化幅度
    
    #截断值取势阱底部时的原子间距离,即rmin=rCut=2^1/6 *sigma
    rCut=math.pow(2.,1./6. * sigma)
    
    #计算在预设密度条件下，粒子所在区域的大小
    """
    三维空间下的：
    desity=粒子数目/体积，
    平衡体系模拟，密度保持不变，粒子数目不变则需要根据密度来更新粒子活动区域的大小，如下:
    """
    region=np.multiply(1./math.pow(density,1./3.),initUcell) #？
    nMol=len(mol)
    
    #速度的变化幅度取决于温度的高低
    velMag = math.sqrt(NDIM * (1. -1. /nMol) * T)

    """
所有初始化计算的工作都集中在以下的函数。
"""
def SetupJob(): 
    global stepCount   #步长计数
    
    stepCount = 0  #初始化步长计数变量的值
    InitCoords()   #初始化坐标
    InitVels()     #初始化速度
    InitAccels()   #初始化加速速度
    AccumProps(0)  #初始化系统属性值（总能量，动能，压强）

"""
代码7 ：InitCoords()，InitVels()，InitAccels()

这里主要是初始化体系粒子的坐标，速度，加速度的函数
"""

# 初始化坐标。
# 这里使用了一个简单的正方形晶格（可以选择不等长的边）,
# 因此，每个元胞只包含一个原子，系统以原点为中心，
def InitCoords():
    c=np.asarray([0.0,0.0,0.0])                       #初始坐标值
    gap=np.divide(region,initUcell)                   #单个粒子所在元胞的大小
    n=0                                               #给粒子计数
    
    #将8000个原子分别放到元胞中（这些元胞向x,y方向扩胞成了region区域）
    for nz in range(0, initUcell[2]):
        for ny in range(0, int(initUcell[1])):
            for nx in range(0, int(initUcell[0])):
                c = np.asarray([nx+0.5, ny+0.5, nz+0.5])
                c = np.multiply(c, gap)
                c = np.add(c, np.multiply(-0.5, region))
                mol[n].pos = c
                n=n+1

            
# 初始化速度。
# 初始速度被设置为固定幅度（velMag）,
# 这取决于温度。在分配了随机速度方向后
# 调整速度方向，以确保质心是静止的，因为系统在不受净合外力作用下，其质心速度应该保持不变,
# 函数vRand作为均匀分布的径向单位向量的来源。
def InitVels():
    global vSum
    vSum=np.zeros(vSum.shape)   #返回特定大小，以0填充新的数组:[0,0]，这里是形状
    
    for n in range(nMol):
        VRand(mol[n].vel)                               #产生随机随机速度向量[x,y,z]
        mol[n].rv=np.multiply(mol[n].vel,velMag)        #根据温度，生成新的速度
        vSum=np.add(vSum,mol[n].vel)                    #质心的速度，系统总速度各质点速度的总和
    
    #调整度方向，以确保质心是静止的,这里取8000个粒子在x,y,z方向的平均速度1/8000*Vsum+每一个原子的速度
    for n in range (nMol):
        mol[n].vel=np.add(mol[n].vel,np.multiply((- 1.0 / nMol),vSum))

        
#初始化加速度。
# 加速度被初始化为零[0,0]
def InitAccels():
    for n in range(nMol):
        mol[n].a=np.zeros(mol[n].a.shape)

1234